## Legacy reporter 


A simple reporter that only saves conformations in a "legacy" format - using polymerutils.save 


In [1]:
import polychrom
import numpy as np 
import warnings
import h5py 
import glob
from polychrom.simulation import Simulation
import polychrom.starting_conformations
import polychrom.forces, polychrom.forcekits
import simtk.openmm 
import os 
import shutil
import polychrom.polymerutils

### Loading reporter and utils from a hdf5_format module 


In [2]:
from polychrom.legacy.legacy_format import LegacyReporter

### Making a simulation and passing a reporter 

In [3]:
%rm  test_legacy/*
data = polychrom.starting_conformations.grow_cubic(10000,30)

"""
Here we created a hdf5Reporter attached to a foler test, and we are saving 5 blocks per file 
(you should probalby use 50 here or 100. 5 is just for a showcase)
"""
reporter = LegacyReporter(folder="test_legacy")


"""
Passing a reporter to the simulation object - many reporters are possible, and more will be added in a future
"""
sim = Simulation(N=10000, error_tol=0.001, collision_rate=0.01, integrator ="variableLangevin", platform="CPU", 
                reporters=[reporter])
sim.set_data(data)
sim.add_force(polychrom.forcekits.polymerChains(sim))
sim._apply_forces()
sim.add_force(polychrom.forces.sphericalConfinement(sim, density=0.1))


for i in range(19):        
    """
    Here we pass two extra records: a string and an array-like object.
    First becomes an attr, and second becomes an HDF5 dataset
    """
    sim.do_block(10, saveExtras={"eggs": "I don't eat green eggs and ham!!!", "spam":[1,2,3]})

"""
Here we are not forgetting to dump the last set of blocks that the reporter has. 
We have to do it at the end of every simulation. 

I tried adding it to the destructor to make it automatic,
but some weird interactions with garbage collection made it not very useable. 
"""
reporter.dump_data()




INFO:root:adding force HarmonicBonds 0
INFO:root:adding force Angle 1
INFO:root:adding force PolynomialRepulsive 2
INFO:root:Particles loaded. Potential energy is 0.050551


Exclude neighbouring chain particles from PolynomialRepulsive
Number of exceptions: 9999


INFO:root:block    1 pos[1]=[13.9 14.2 14.3] dr=0.21 t=0.9ps kin=8.20 pot=3.92 Rg=11.076 dt=24.2fs dx=15.46pm 
INFO:root:block    2 pos[1]=[13.9 14.2 14.3] dr=0.17 t=1.2ps kin=3.82 pot=8.83 Rg=11.077 dt=20.6fs dx=8.99pm 
INFO:root:block    3 pos[1]=[13.7 14.2 14.3] dr=0.09 t=1.4ps kin=7.74 pot=4.96 Rg=11.079 dt=24.7fs dx=15.36pm 
INFO:root:block    4 pos[1]=[13.7 14.2 14.3] dr=0.14 t=1.6ps kin=6.27 pot=6.44 Rg=11.083 dt=22.3fs dx=12.45pm 
INFO:root:block    5 pos[1]=[13.8 14.2 14.3] dr=0.11 t=1.8ps kin=6.73 pot=5.95 Rg=11.087 dt=22.0fs dx=12.76pm 
INFO:root:block    6 pos[1]=[13.9 14.2 14.4] dr=0.12 t=2.0ps kin=6.77 pot=5.88 Rg=11.092 dt=22.0fs dx=12.80pm 
INFO:root:block    7 pos[1]=[13.7 14.1 14.3] dr=0.12 t=2.3ps kin=6.75 pot=5.88 Rg=11.097 dt=22.0fs dx=12.78pm 
INFO:root:block    8 pos[1]=[13.6 14.1 14.3] dr=0.12 t=2.5ps kin=7.20 pot=5.40 Rg=11.102 dt=22.0fs dx=13.20pm 
INFO:root:block    9 pos[1]=[13.7 14.1 14.3] dr=0.12 t=2.7ps kin=6.71 pot=5.87 Rg=11.108 dt=22.0fs dx=12.74pm 
IN

### This is a list of files created in the trajectory folder

In [4]:
!ls -la test_legacy

total 3732
drwxrwxr-x 2 magus magus   4096 Sep  2 15:34 .
drwxrwxr-x 5 magus magus   4096 Sep  2 15:28 ..
-rw-rw-r-- 1 magus magus 200606 Sep  2 15:34 block0.dat
-rw-rw-r-- 1 magus magus 200603 Sep  2 15:34 block10.dat
-rw-rw-r-- 1 magus magus 200587 Sep  2 15:34 block11.dat
-rw-rw-r-- 1 magus magus 200586 Sep  2 15:34 block12.dat
-rw-rw-r-- 1 magus magus 200576 Sep  2 15:34 block13.dat
-rw-rw-r-- 1 magus magus 200596 Sep  2 15:34 block14.dat
-rw-rw-r-- 1 magus magus 200610 Sep  2 15:34 block15.dat
-rw-rw-r-- 1 magus magus 200605 Sep  2 15:34 block16.dat
-rw-rw-r-- 1 magus magus 200592 Sep  2 15:34 block17.dat
-rw-rw-r-- 1 magus magus 200601 Sep  2 15:34 block18.dat
-rw-rw-r-- 1 magus magus 200605 Sep  2 15:34 block1.dat
-rw-rw-r-- 1 magus magus 200573 Sep  2 15:34 block2.dat
-rw-rw-r-- 1 magus magus 200590 Sep  2 15:34 block3.dat
-rw-rw-r-- 1 magus magus 200601 Sep  2 15:34 block4.dat
-rw-rw-r-- 1 magus magus 200615 Sep  2 15:34 block5.dat
-rw-rw-r-- 1 magus magus 200604 Sep  2 15:34 

In [5]:
polychrom.polymerutils.load("test_legacy/block5.dat")

array([[13.889, 14.204, 14.378],
       [12.747, 13.854, 14.075],
       [12.05 , 13.958, 13.383],
       ...,
       [15.048, 15.021, 12.98 ],
       [14.19 , 15.234, 12.93 ],
       [14.022, 14.294, 13.121]])

In [7]:
sim.addForce

<bound method Simulation.addForce of <polychrom.simulation.Simulation object at 0x7f1c3e9ff590>>